# Transfer Learning with TensorFlow Part 3: Scaling Up (Food Vision)

We've seen the power of transfer learning feature extraction and fine-tuning, now it's time to scale up to all of the classes in Food101 (101 classes of food).

Our goal is to beat the original Food101 paper with 10% of the training data (leveraging the power of deep learning)

Our baseline to beat is 50.76% accuracy across 101 classes

## Creating helper functions

In previous notebooks, we've created a series of helper function to do different tasks, let's download and import them

In [ ]:
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, compare_historys, walk_through_dir

##  101 Food Classes: Working with less data

Our goal is to beat the original Food101 paper with 10% of the training data, so let's download it.

The data we're downloading comes from the original Food101 dataset but has been preprocessed using the `image_data_modification` notebook

In [ ]:
# Unzip the data
unzip_data("101_food_classes_10_percent.zip")

In [ ]:
# Setup directories
train_dir="101_food_classes_10_percent/train/"
test_dir="101_food_classes_10_percent/test/"

In [ ]:
# How many images/class in our dataset
walk_through_dir("101_food_classes_10_percent")

In [ ]:
# Setup our data loaders 
import tensorflow as tf
IMG_SIZE=(224,224)
BATCH_SIZE=32

train_data_all_10_percent=tf.keras.preprocessing.image_dataset_from_directory(train_dir,
                                                                              image_size=IMG_SIZE,
                                                                              batch_size=BATCH_SIZE,
                                                                              label_mode="categorical")

test_data=tf.keras.preprocessing.image_dataset_from_directory(test_dir,
                                                              image_size=IMG_SIZE,
                                                              batch_size=BATCH_SIZE,
                                                              label_mode="categorical",
                                                              shuffle=False) # don't shuffle test data for prediction analysis                                                                                                                                                                                                                                                         

## Train a big model with transfer learning on 10% of 101 food classes

Here are the steps we're going to take:
* Create a `ModelCheckpoint` callback
* Create a data augmentation layer to build data augmentation right into the model
* Build a headless (no top layers) Functional EfficientNetB0 backboned-model (we'll create our own output layer)
* Compile our model
* Feature extract for 5 full passes (5 epochs on the train dataset and validate on 15% of the test data, to save epoch time)


In [ ]:
# Create checkpoint callback
checkpoint_path="101_food_classes_10_percent_data_model_checkpoint"
checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                       save_weights_only=True,
                                                       monitor="val_accuracy",
                                                       save_best_only=True)

In [ ]:
# Create data augmentation layer to incorporate it right into the model
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing

data_augmentation=tf.keras.Sequential([
    preprocessing.RandomFlip("horizontal"),
    preprocessing.RandomZoom(0.2),
    preprocessing.RandomHeight(0.2),
    preprocessing.RandomWidth(0.2)
],name="data_augmentation")

In [ ]:
# Build a headless model (no top layers) Functional EfficientNetB0 model

# Input shape of the data
input_shape=(224,224,3)

# Create a frozen base model
base_model=tf.keras.applications.EfficientNetB0(include_top=False)
base_model.trainable=False

# Create layers
inputs=layers.Input(shape=input_shape,name="input_layer")
x=data_augmentation(inputs)
x=base_model(x,training=False)
x=layers.GlobalAveragePooling2D(name="global_avg_pool_layer")(x)
outputs=layers.Dense(len(train_data_all_10_percent.class_names),activation="softmax",name="output_layer")(x)

# Construct model
model=tf.keras.Model(inputs,outputs)        

In [ ]:
# Get a summary of the model we created
model.summary()

In [ ]:
# Compile the model
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(),
              metrics=["accuracy"])

# Fit the model 
history_all_classes_10_percent = model.fit(train_data_all_10_percent,
                                           epochs=5,
                                           validation_data=test_data,
                                           validation_steps=int(0.15*len(test_data)), # validate on only 15% of the data
                                           callbacks=[checkpoint_callback])


In [ ]:
# Evaluate on the whole test dataset
feature_extraction_results=model.evaluate(test_data)

In [ ]:
# Let's look at the loss curves
plot_loss_curves(history_all_classes_10_percent)

**Note**: It appears that our model is overfitting (performing too well on the training data and not generalizing to unseen data)

## Fine-tuning

In [ ]:
# Unfreeze all of the layers in the base model
base_model.trainable=True

# Refreeze every layer but the last 5
for layer in base_model.layers[:-5]:
    layer.trainable=False

In [ ]:
# Recompile the model with lower learning_rate
model.compile(loss="categorical_crossentropy",
              optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), # learning rate lowered by 10x
              metrics=["accuracy"])

In [ ]:
# What layers in the model are trainable
for layer in model.layers:
    print(layer.name,layer.trainable)

In [ ]:
# Check which layers are trainabale in our base model
for layer_number,layer in enumerate(base_model.layers):
    print(layer_number,layer.name,layer.trainable)

In [ ]:
# Fine tune for another 5 epochs
# model has already done 5 epochs, total number of epochs we're after (5+5 = 10)
fine_tune_epochs = 10

# Refit the model
history_all_classes_10_percent_fine_tune = model.fit(train_data_all_10_percent,
                                                     epochs=fine_tune_epochs,
                                                     initial_epoch=history_all_classes_10_percent.epoch[-1],
                                                     validation_data=test_data,
                                                     validation_steps=int(0.15*len(test_data)))


In [ ]:
# Evaluate on the whole test data
all_classes_10_percent_fine_tune_results=model.evaluate(test_data)

In [ ]:
compare_historys(history_all_classes_10_percent,history_all_classes_10_percent_fine_tune)

## Saving and loading our model

To use our model in an external application, we'll need to save it and export it somewhere

In [ ]:
# Save our fine-tuned model
model.save("G:/UMass/Courses/Year_4 (2022)/TensorFlowDeveloperLocal/TensorFlow-Developer-Course/101_food_classes_10_percent_saved_big_dog_model")

In [ ]:
# Load an evaluated model
loaded_model=tf.keras.models.load_model("G:/UMass/Courses/Year_4 (2022)/TensorFlowDeveloperLocal/TensorFlow-Developer-Course/101_food_classes_10_percent_saved_big_dog_model")

In [ ]:
# Evaluate loaded model and compare performance to pre-saved model
loaded_model_results=loaded_model.evaluate(test_data)

In [ ]:
# The results from oour loaded_model (above) should be very similar to the results below
all_classes_10_percent_fine_tune_results

## Evaluating the performance of the big model across all different classes

Let's make some predictions, visualize them, and then later find out which predictions were the "most" wrong

In [ ]:
# Unzip the data
unzip_data("06_101_food_class_10_percent_saved_big_dog_model.zip")

In [ ]:
# Load in saved model
model=tf.keras.models.load_model("G:/UMass/Courses/Year_4 (2022)/TensorFlowDeveloperLocal/TensorFlow-Developer-Course/06_101_food_class_10_percent_saved_big_dog_model")

In [ ]:
# Evaluate the loaded model
results_downloaded_model=model.evaluate(test_data)

## Making predictions with our trained model

In [ ]:
# Making predictions with model
preds_probs = model.predict(test_data,verbose=1) # set verbosity to see how long is left